In [6]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Math
import os
print(os.getcwd())  # Print the current working directory
print(os.listdir()) # List all files in the current directory
def load_joint_data(npy_filename):
    """
    Loads joint position (q), velocity (qd), and acceleration (qdd) from a CSV file.

    """
    data = np.load(npy_filename, allow_pickle=True)
    data = data[0] if isinstance(data[0], list) else data
    q = np.vstack((data[0])).T
    qd = np.vstack((data[1])).T 
    qdd = np.vstack((data[2])).T 
    return q, qd, qdd

q_csv, qd_csv, qdd_csv = load_joint_data('../data/simplePendLE.npy')

a:\uni\masters nu\RA\pythonCode\rnea
['double3Dpendthetas.npy', 'Figure_1.png', 'Figure_2.png', 'LE_matlab.m', 'LE_wolfram.nb', 'NE_matlab.m', 'rnea.py', 'rnea_old.py', 'rnea_single_link _simplesty.py', 'rnea_single_link.py', 'rnea_single_sym.ipynb', 'simplePendLE.mat']


In [8]:

# Define symbolic variables
t = sp.Symbol('t', real=True)  # Time variable
q = sp.Function('q')(t)        # Joint position as a function of time
qd = q.diff(t)                 # Joint velocity (first derivative)
qdd = qd.diff(t)               # Joint acceleration (second derivative)

# Parameters
m = sp.Symbol('m', positive=True)       # Mass
l = sp.Symbol('l', positive=True)       # Length
g = sp.Symbol('g', positive=True)       # Gravity
b = sp.Symbol('b', real=True)           # Damping coefficient
I = sp.Symbol('I', positive=True)       # Moment of inertia

# Define symbolic functions for external torque
tau_ext = sp.Function('tau_ext')(t)

# Symbolic implementation of rotation matrix
def rotation_matrix_sym(theta, alpha):
    return sp.Matrix([
        [sp.cos(theta), -sp.cos(alpha) * sp.sin(theta), sp.sin(alpha) * sp.sin(theta)],
        [sp.sin(theta), sp.cos(alpha) * sp.cos(theta), -sp.sin(alpha) * sp.cos(theta)],
        [0, sp.sin(alpha), sp.cos(alpha)]
    ])

def rotation_matrix_to_base_sym(theta):
    # For a single link pendulum, this is simply the rotation matrix
    return sp.Matrix([
        [sp.cos(theta), -sp.sin(theta), 0],
        [sp.sin(theta), sp.cos(theta), 0],
        [0, 0, 1]
    ])

def s_vector_sym(theta, length):
    # Center of mass position vector (relative to joint)
    return sp.Matrix([
        [-length/2 * sp.cos(theta)],
        [-length/2 * sp.sin(theta)],
        [0]
    ])

def p_star_vector_sym(theta, length):
    # End-point position vector
    return sp.Matrix([
        [length * sp.cos(theta)],
        [length * sp.sin(theta)],
        [0]
    ])

# Symbolic implementation of RNEA for a single pendulum
def symbolic_rnea_single_pendulum():
    print("Deriving symbolic equations for a single pendulum using RNEA...")
    
    # Define gravity vector
    gravity = sp.Matrix([0, -g, 0])
    
    # Joint type (1 for revolute)
    j_type = 1
    
    # Rotation matrix for the link
    R = rotation_matrix_sym(q, 0)
    R_base = rotation_matrix_to_base_sym(q)
    
    # Position vectors
    p_star_vec = p_star_vector_sym(q, l)
    s_bar_vec = s_vector_sym(q, l)
    
    # 1. Forward recursion
    # Angular velocity and acceleration
    omega = sp.Matrix([0, 0, qd])
    omegad = sp.Matrix([0, 0, qdd])
    
    # Linear acceleration (including gravity)
    vd = gravity
    
    # Center of mass acceleration
    a_c = omegad.cross(R_base * s_bar_vec) + omega.cross(omega.cross(R_base * s_bar_vec)) + vd
    
    # 2. Backward recursion
    # Compute force: F = m * a_c
    F = m * a_c
    
    # Compute torque: N = I * ω̇ + ω × (I * ω)
    # For a simple pendulum with inertia about z-axis
    I_tensor = sp.Matrix([
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, I]
    ])
    
    N = R_base * I_tensor * R_base.T * omegad + omega.cross(R_base * I_tensor * R_base.T * omega)
    
    # Link force and torque
    f = F
    n_torque = (R_base * p_star_vec + R_base * s_bar_vec).cross(F) + N
    
    # Joint torque (for revolute joint)
    z_axis = sp.Matrix([0, 0, 1])
    tau = n_torque.dot(z_axis) + b * qd
    
    # Simplify the expression
    tau_simplified = sp.simplify(tau)
    
    # Equation of motion: tau_ext = tau
    eom = sp.Eq(tau_ext, tau_simplified)
    
    return eom, tau_simplified

# Execute the symbolic derivation
eom, tau_expr = symbolic_rnea_single_pendulum()

# Display the equation of motion
print("\nEquation of Motion (EOM):")
display(Math(sp.latex(eom)))

print("\nExpanded expression for required torque:")
display(Math(sp.latex(tau_expr)))

# Rearrange to get the standard form of pendulum dynamics
# tau_ext - other terms = I*qdd
qdd_expr = sp.solve(sp.Eq(tau_ext - tau_expr + I*qdd, 0), qdd)[0]
print("\nExpression for angular acceleration (qdd):")
display(Math(sp.latex(sp.Eq(qdd, qdd_expr))))

# Now let's define a specific external torque function to compare with the original code
print("\nSpecifying external torque as a sine function:")
tau_ext_func = 2 * sp.sin(0.5 * t)
display(Math(sp.latex(sp.Eq(tau_ext, tau_ext_func))))

# Substitute the external torque into the equation of motion
eom_with_input = eom.subs(tau_ext, tau_ext_func)
print("\nEquation of motion with the specified input torque:")
display(Math(sp.latex(eom_with_input)))

# Define numerical values for simulation
print("\nNumerical simulation with specific parameter values:")
param_values = {
    m: 1.0,
    l: 1.0,
    g: 9.81,
    I: 0.1,
    b: 0
}

# Substitute the parameter values
tau_numeric = tau_expr.subs(param_values)
print("\nTorque expression with numerical values:")
display(Math(sp.latex(tau_numeric)))

# Define a function to convert symbolic expressions to numpy functions
def sym_to_numpy(expr, vars_list):
    return sp.lambdify(vars_list, expr, "numpy")

# Create numpy functions for numerical simulation
tau_func = sym_to_numpy(tau_numeric, [q, qd, qdd])

# For comparison with the original code, let's also define the input torque function
def tau_input(t_val):
    return np.array([2 * np.sin(0.5 * t_val), 1.5 * np.cos(0.5 * t_val)])

# Now let's simulate a simple pendulum response to visualize
print("\nSimulating the pendulum response to the input torque...")

# For a more comprehensive analysis, we would solve the ODE here
# But for simplicity, let's just plot the torque for given motion

# Define a sample motion pattern
def sample_q(t_val):
    return np.sin(t_val) + 0.5 * np.cos(0.5 * t_val)

def sample_qd(t_val):
    return np.cos(t_val) - 0.25 * np.sin(0.5 * t_val)

def sample_qdd(t_val):
    return -np.sin(t_val) - 0.125 * np.cos(0.5 * t_val)

# Time steps
time = np.linspace(0, 10, 1000)

# Compute torques for the sample motion
computed_torques = []
input_torques = []
i = 0
for t_val in time:
    # Sample motion values
    q_val = q_csv[i]
    qd_val = qd_csv[i]
    qdd_val = qdd_csv[i]
    i+=1
    # Computed torque required for this motion
    torque = tau_func(q_val, qd_val, qdd_val)
    computed_torques.append(torque)
    
    # Input torque from the function
    input_torque = tau_input(t_val)[0]  # First element only
    input_torques.append(input_torque)

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(time, computed_torques, label='Computed Torque (RNEA)')
plt.plot(time, input_torques, label='Input Torque')
plt.plot(time, [sample_qd(t) for t in time], label='qd (Velocity)')
plt.xlabel('Time (s)')
plt.ylabel('Torque (Nm) / Velocity (rad/s)')
plt.title('Pendulum Dynamics: Torque and Velocity')
plt.legend()
plt.grid(True)
plt.show()

# Symbolic derivation for a double pendulum
# This would extend the above analysis but is more complex
print("\nFor extending to a double pendulum, we would:")
print("1. Define additional symbolic variables for the second joint")
print("2. Extend the rotation matrices and position vectors")
print("3. Follow the same RNEA approach with recursive forward and backward passes")
print("4. Derive the coupled equations of motion")

Deriving symbolic equations for a single pendulum using RNEA...

Equation of Motion (EOM):


<IPython.core.display.Math object>


Expanded expression for required torque:


<IPython.core.display.Math object>


Expression for angular acceleration (qdd):


...

<IPython.core.display.Math object>


Numerical simulation with specific parameter values:

Torque expression with numerical values:


<IPython.core.display.Math object>


Simulating the pendulum response to the input torque...


IndexError: index 1 is out of bounds for axis 0 with size 1